# Circle shenanningans

In [2]:
# imports 


import math
import numpy as np

In [25]:

def get_GoalPos( proj, radius, center_circle, angle_increment= 1):

    """
    A function that returns the goal position of the Ant. 
        The goal position is computed by getting the angle between the lines (center_circle, ant_proj_line) and (center_circle, positive_XAxis),
            increasing the angle by angle_incremet, computing the sin and cos of the new angle, multiplying them by radius and adding center_circle.
                The resulting coordinates are those of goal_pos - a step along the circle in the counterclockwise direction
 
    Parameters:
    -----------
    proj: np array size (2,): the x, y coordinates of the proj_lineection of the Ant.
    danger_zone: float between 0 and 1, default = 0.95.
        This is a fast and dirty implementation, so I have to micromanage at parts. When the Ant dets close to the intersections of the circle and the x axis, 
            the y coordinate of goal_pos must change its sign.
    angle_incremet: float, the increment(degrees) added to the new angle used to compute goal_pos
    
    Returns:
    --------
    np array size (2,) = the x, y coordinates of the goal position

    """


    import math
    import numpy as np

    proj_line =  np.array([center_circle, proj ]).tolist()    # transforming np arrays to embedded lists, since np's array opperations perturb the algorithm
    x_axis_line = np.array([ center_circle, [ center_circle[0] + radius, center_circle[1]] ] ).tolist()

    try:
        slope1 = (x_axis_line[1][1] - x_axis_line[0][1]) / (x_axis_line[1][0] - x_axis_line[0][0])
    except ZeroDivisionError:
        slope1 = (x_axis_line[1][1] - x_axis_line[0][1]) / 1e-8

    try:
        slope2 = (proj_line[1][1] - proj_line[0][1]) / (proj_line[1][0] - proj_line[0][0])
    except ZeroDivisionError:
        slope2 = (proj_line[1][1] - proj_line[0][1]) / 1e-8

    # Calculate the angle between the two lines
    angle_rad = math.atan2(slope2 - slope1, 1 + slope1 * slope2)

    new_angle_rad = math.radians( math.degrees(angle_rad) + angle_increment )

    new_sin, new_cos = math.sin(new_angle_rad), math.cos(new_angle_rad)

    print(center_circle)
    
    print(np.array( [new_sin, new_cos] ) * radius)
    goal_pos = ( np.array( [new_sin, new_cos] ) * radius ) 

    return goal_pos

In [19]:
a = np.array([1,2])
b = np.array([3,4])
print(a+b)

[4 6]


In [36]:
print( -1.00000000e+01)

-10.0


In [26]:
center_circle = np.array([10, 0])
proj =   np.array([10, 10])
radius = 10

get_GoalPos(proj= proj, center_circle= center_circle, radius= radius, angle_increment= 90)

[10  0]
[ 1.00000021e-08 -1.00000000e+01]


array([ 1.00000021e-08, -1.00000000e+01])

In [25]:
print(math.sin(90))

0.8939966636005579


# sb3 testing

In [1]:
# example basic usage from here:https://stable-baselines3.readthedocs.io/en/master/guide/examples.html


import gym

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

In [5]:
# Create environment
env = gym.make("CartPole-v1")

In [7]:
# Instantiate the agent
model = DQN("MlpPolicy", env = "CartPole-v1", verbose=1)
# Train the agent and display a progress bar
model.learn(total_timesteps=3, progress_bar=True)
# Save the agent
model.save("dqn_lunar")
del model  # delete trained model to demonstrate loading


Using cpu device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.

In [9]:
import gym
env = gym.make("CartPole-v1")

observation, info = env.reset(seed=42)
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()

In [ ]:
# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = DQN.load("dqn_lunar", env=env)


In [ ]:
# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)


In [ ]:
# Enjoy trained agent
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()
